In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.utils.data.dataloader
from tqdm import tqdm
import numpy as np

# Comprobamos si se puede usar la gráfica para realizar los procesos más rápido.

In [ ]:
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device: {torch.cuda.current_device()}")
print(f"Name of current CUDA device: {torch.cuda.get_device_name(cuda_id)}")
device="cuda" if torch.cuda.is_available() else "cpu" 
torch.backends.cudnn.benchmark = True

## Si vemos que nos devuelve CPU es o bien porque no tenemos instalado CUDA o pq nuestro dispositivo no lo soporta.
Así pues vamos a proceder a instalarlo.

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

# Cargamos el dataset (preparamos los datos)

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((128, 128)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    torchvision.traq
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataloader = {
    'train': torch.utils.data.DataLoader(torchvision.datasets.Food101(r"D:\datasets", split="train", download=True, transform=transform), batch_size=128, shuffle=True, pin_memory=True, num_workers=4),
    'test': torch.utils.data.DataLoader(torchvision.datasets.Food101(r"D:\datasets", split="test", transform=transform), batch_size=128, shuffle=True, pin_memory=True, num_workers=4)
}

dataloader["train"]

# Definimos el modelo de nuestra red nueronal.

In [4]:
def block(c_in, c_out, k=3, p=1, s=1, pk=2, ps=2):
    return torch.nn.Sequential(
        torch.nn.Conv2d(c_in, c_out, k, padding=p, stride=s),
        torch.nn.BatchNorm2d(c_out),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(pk, stride=ps)
    )

class FoodCNN(torch.nn.Module):
  def __init__(self, n_channels=3, n_outputs=101):
    super().__init__()
    self.conv1 = block(n_channels, 64)
    self.conv2 = block(64, 128)
    self.conv3 = block(128, 256)
    self.conv4 = block(256, 512)
    self.conv5 = block(512,512)
    self.fc1 = torch.nn.Linear(512*4*4, 512) # Lo cambiamos de 1024 a 512 para probar eficiencia.
    self.fc2 = torch.nn.Linear(512, n_outputs)
    self.dropout = torch.nn.Dropout(0.7) # Lo augmentamos de 0.5 a 0.6-0.7 para tratar de reducir el sobreajuste.

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.conv4(x)
    x = self.conv5(x)
    x = x.view(x.shape[0], -1)
    x = self.dropout(self.fc1(x))  # Aplicar Dropout en esta capa
    x = self.fc2(x)
    return x


# Entrenamos el modelo.

In [5]:
def train(model, dataloader, optimizer, scheduler, epochs=1, start_epoch=1):
    model.to(device)
    scaler = torch.amp.GradScaler(device='cuda')
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(start_epoch, epochs+1):
        model.train()
        train_loss, train_acc = [], []
        bar = tqdm(dataloader['train'], desc=f"Epoch {epoch}/{epochs} - Training")

        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                y_hat = model(X)
                loss = criterion(y_hat, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            train_loss.append(loss.item())
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            train_acc.append(acc)
            bar.set_description(f"loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")
        
        bar = tqdm(dataloader['test'], desc=f"Epoch {epoch}/{epochs} - Validation")
        val_loss, val_acc = [], []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
                val_acc.append(acc)
                bar.set_description(f"val_loss {np.mean(val_loss):.5f} val_acc {np.mean(val_acc):.5f}")
        
        # Scheduler step
        mean_val_loss = np.mean(val_loss)
        mean_val_acc = np.mean(val_acc)
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {mean_val_loss:.5f} acc {np.mean(train_acc):.5f} val_acc {mean_val_acc:.5f}")

        # Llamada correcta al scheduler con la métrica `mean_val_loss`
        scheduler.step(mean_val_loss)

        # print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f} acc {np.mean(train_acc):.5f} val_acc {np.mean(val_acc):.5f}")

In [ ]:
model = FoodCNN()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4) # Probamos con SGD en vez de Adam
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)  # Reducción de LR cada 3 épocas
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)


# 3. Continuar Entrenamiento
# Ajusta la función de entrenamiento para usar `start_epoch`
start_epoch, additional_epochs = 1, 2  # La siguiente época después de las 7 completadas = 8
# additional_epochs = 1  # Número de épocas adicionales
total_epochs = start_epoch + additional_epochs - 1

# Ejecutar entrenamiento desde la época 8 hasta la 17
train(model, dataloader, optimizer, scheduler, epochs=total_epochs, start_epoch=start_epoch)

## Guardamos el estado actual del modelo (opcional).

In [43]:
PATH = r"C:\...\checkpoints\checkpoint_29_10_24_v5.pt"
torch.save({
    'epoch': total_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict()
}, PATH)

## Cargamos modelo (opcional).

In [ ]:
PATH = r"C:\...\checkpoints\checkpoint_29_10_24_v5.pt"
model = FoodCNN()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4) # Probamos con SGD en vez de Adam
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)  # Reducción de LR cada 3 épocas
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
checkpoint = torch.load(PATH, map_location=device, weights_only=True)  # Usa weights_only para mayor segurida. Asegura que los parámetros se carguen en `cuda`
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
num_epochs = checkpoint["epoch"]
print(f"Estructura del modelo: \n\n{model.eval()}")
print(f"Estructura del checkpoint: \n\n{checkpoint.keys()}")
print(f"Número de epochs: {num_epochs}\n\n")

## Probamos que no ha sufrido ninguna alteración la copia (opcional).

In [12]:
def evaluate(model, dataloader):
    model.eval()
    model.to(device)
    bar = tqdm(dataloader['test'])
    acc = []
    with torch.no_grad():
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            y_hat = model(X)
            acc.append((y == torch.argmax(y_hat, axis=1)).sum().item() / len(y))
            bar.set_description(f"val_acc {np.mean(acc):.5f}")

In [ ]:
evaluate(model, dataloader)

# Proseguir con el entrenamiento, desde un checkpoint.

In [ ]:
PATH = r"D:\...\checkpoint_29_10_24_v5.pt"

# 1. Inicializar el modelo y otros componentes
model = FoodCNN().to(device)  # Mueve el modelo a `cuda`
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

# 2. Cargar el checkpoint
checkpoint = torch.load(PATH, map_location=device, weights_only=True)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
start_epoch = checkpoint["epoch"]  # Aquí simplemente asignamos el valor

# Asegurarse de que todos los estados del optimizador estén en `cuda`
for state in optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.to(device)

# 3. Continuar Entrenamiento
start_epoch += 1  # La siguiente época después de las completadas
additional_epochs = 5
total_epochs = start_epoch + additional_epochs - 1

# Ejecutar entrenamiento
train(model, dataloader, optimizer, scheduler, epochs=total_epochs, start_epoch=start_epoch)


# Prueba del funcionamiento.

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt

# Definir las transformaciones
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Cargar los nombres de las categorías
def load_class_names():
    with open(r"D:\datasets\food-101\meta\classes.txt") as f:
        classes = [line.strip() for line in f.readlines()]
    return classes

# Función para predecir la categoría de una imagen y mostrar el porcentaje de certeza
def predict_image(image_path, model, device, class_names):
    # Cargar la imagen
    image = Image.open(image_path).convert('RGB')
    
    # Aplicar las transformaciones
    transformed_image = transform(image).unsqueeze(0)  # Añadir una dimensión para el batch
    
    # Mover la imagen al dispositivo (GPU o CPU)
    transformed_image = transformed_image.to(device)
    
    # Poner el modelo en modo de evaluación
    model.eval()
    
    # Desactivar el cálculo de gradientes
    with torch.no_grad():
        # Realizar la predicción
        output = model(transformed_image)
        probabilities = torch.nn.functional.softmax(output, dim=1)  # Convertir la salida a probabilidades
        confidence, predicted = torch.max(probabilities, 1)
    
    # Obtener la categoría y la certeza de la predicción
    predicted_category = class_names[predicted.item()]
    confidence_percentage = confidence.item() * 100
    
    # Mostrar la imagen con la categoría y el porcentaje de certeza
    plt.imshow(image)
    plt.title(f'Categoría predicha: {predicted_category}\nConfianza: {confidence_percentage:.2f}%')
    plt.axis('off')
    plt.show()

    # También devuelve los resultados en caso de que quieras hacer algo más con ellos
    return predicted_category, confidence_percentage

# Ejemplo de uso
model = FoodCNN()
PATH = r"C:\...\checkpoints\checkpoint_29_10_24_v5.pt"
checkpoint = torch.load(PATH, map_location=device, weights_only=True)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

# Cargar los nombres de las categorías
class_names = load_class_names()

# Ruta de la imagen de prueba
image_path = r"C:\...\images-for-test\waffles-1.jpeg"
predicted_category, confidence = predict_image(image_path, model, device, class_names)
# print(f'La categoría predicha es: {predicted_category} con un {confidence:.2f}% de certeza')


# Preparación para la website.

In [ ]:
# Requiere de una precarga del modelo.

model.export()